# Data Science Project: Flight Delay Prediction: Small data tryout

Note: this is a part of an exercise in CME 250A: Machine Learning with Big data

## Stage 1: Ask a question

My objective is to predict if a flight arrival is going to be delayed or not.

The performance is measured by AUC since it is a classfication problem.

## Stage 2: Set the environment up and get data

In this section, we need only local h2o. Please install h2o for desktop and h2o for Python. See http://www.h2o.ai/download/h2o/choose

This is how to launch h2o on a local machine.

1. After installing h2o for Desktop and h2o for Python, open Git Shell window. Go to your h2o-x.x.x.x directory (likely to be your Home directory (Admin or your user name). Useful command is $\texttt{cd h2o-3.8.2.3}$. 

2. Launch and specify the working memory (in this case 4GB) using command $\texttt{java -Xmx4g -jar h2o.jar}$. 

3. To see the web interface of h2o. Go to URL: $\texttt{http://localhost:54321/flow/index.html }$

In [222]:
# Python mode
import os
import time

os.getcwd() #Return a string representing the current working directory

'C:\\Users\\Admin\\Documents\\GitHub\\DataSciencePortal\\DataScienceTemplateAndProjects'

In [223]:
import h2o
#Connect to h2o
h2o.init()

H2O cluster uptime:,2 hours 9 minutes 3 seconds 561 milliseconds
H2O cluster version:,3.8.2.2
H2O cluster name:,Admin
H2O cluster total nodes:,1
H2O cluster total free memory:,1.74 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


I get data by setup Imports and Variables below. Note that the directory is different from the big data version. This is an exploratory phase before doing the real big data job.

In [224]:
# Load the weather data. We need it from S3 amazon set up for CME250A class.

#path for a small flight data
path = "http://stanford-cme250a.s3.amazonaws.com/allyears2k.csv"

print("Import and Parse flight data")
data = h2o.import_file(path=path)


Import and Parse flight data

Parse Progress: [##################################################] 100%


In [309]:
# Load the weather data. We need it from S3 amazon set up for CME250A class.

#Path for a small weather data. This is a small file that I directly download from S3 browser beforehand.
#path = "C:/Users/Tee/Desktop/CME250A/weather/"
path = "C:/Users/Admin/Desktop/CME250A/weather/"

weather_years = [path+"Xheader.csv",
                 path+"X2008.csv"]

print("Import and Parse weather data")
wthr = h2o.import_file(path=weather_years)

Import and Parse weather data

Parse Progress: [##################################################] 100%


We need to reformat this weather link file to work with the cluster. I don't know why it did not work in the first place. It is probably related to the double quote format of CSV file. A quick trick is to remove everything except two columns we need: iata_ref and maslib. Do it and save it as a new file called $\texttt{master-location-identifier-database-20130801-reformatted.csv}$

In [226]:
# Load the link data. The conversion between an airport name and a 6-digit Station code. This is local file. 
# This is a local file but downloadable from http://weather.noaa.gov/tg/site.shtml

airport_weather_link = h2o.import_file(path="C:\Users\Admin\Desktop\CME250A\weather\master-location-identifier-database-20130801-reformatted.csv")
#airport_weather_link = h2o.import_file(path="C:\Users\Tee\Desktop\CME250A\weather\master-location-identifier-database-20130801-reformatted.csv")


Parse Progress: [##################################################] 100%


## Stage 3: Explore the data

Explore, Visualize, Clean, Transform, Feature engineering

### Flight data

In [227]:
#Check the size and summary of data
data.summary()
print data.shape

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,17.0,14.0,-63.0,-16.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1997.5,1.40909090909,14.6010732639,3.82061485288,1345.84666138,1313.22286143,1504.63413038,1485.28916731,NaN,818.842989677,NaN,124.814529135,125.021562607,114.316111091,9.31711193698,10.0073906556,NaN,NaN,730.182190565,5.38136805953,14.1686341847,0.0246941652645,NaN,0.00247851198326,4.04780029106,0.289376469271,4.85503190418,0.0170155602821,7.62006045002,0.555755150302,0.525057983537
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,9.0,3949.0,3500.0,475.0,437.0,402.0,475.0,473.0,131.0,133.0,3365.0,128.0,254.0,1.0,3.0,1.0,369.0,201.0,323.0,14.0,373.0,1.0,1.0
sigma,6.34436090171,1.87471137134,9.17579042586,1.90501311913,465.340899124,476.251139993,484.347487904,492.750434123,NaN,777.404369164,NaN,73.9744416606,73.40159463,69.6363295151,29.8402219624,26.4388090429,NaN,NaN,578.43800823,4.20197993986,9.9050857472,0.155193141358,NaN,0.0497234872189,16.2057299045,4.41677989873,18.6197762215,0.403940182102,23.4875658741,0.496887288343,0.499377380318
zeros,0,0,0,0,0,569,0,569,724,0,2,0,0,0,1514,6393,59,172,0,623,557,42892,81,43869,7344,8840,7388,8914,7140,19537,20887
missing,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


(43978, 31)


General theme: specify data types correctly, do some conversion, and get rid of bad and "too good" predictors. We also build a new feature called 'Date' to merge data in the future.

Note that CRS stands for Computer Reservation System. Using actual information in comparison with CRS information is kind of cheating. So remove actual information.

In [228]:
#Year, Month, DayOfMonth, integer is good. Leave it as it is. 

#DayOfWeek should be treated as enum instead of int
data['DayOfWeek'] = data['DayOfWeek'].asfactor()

In [229]:
#All Time should be converted to minutes (1 = 0*60+ 1, 2359 = 25*60+39)

data = data.drop("DepTime")
data['CRSDepTime'] = (data['CRSDepTime']/100).floor()*60 + (data['CRSDepTime']%100)
data = data.drop("ArrTime")
data['CRSArrTime'] = (data['CRSArrTime']/100).floor()*60 + (data['CRSArrTime']%100)

In [230]:
#UniqueCarrier is good.

#FlightNum should be treated as enum intead of int
data['FlightNum'] = data['FlightNum'].asfactor()

In [231]:
#ActualElapsedTime, CRSElapseTime, AirTime: keep only CRSElapseTime.
data = data.drop("ActualElapsedTime")
data = data.drop("AirTime")

In [232]:
#ArrDelay, DepDelay are "too good" predictors. Drop it.
data = data.drop("ArrDelay").drop("DepDelay")

#Origin and Dest are good.

In [233]:
#Cancelled, CancellationCode, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay are "too good" predictors. Drop it.
data = data.drop("Cancelled").drop("CancellationCode").drop("Diverted").drop("CarrierDelay")\
    .drop("WeatherDelay").drop("NASDelay").drop("SecurityDelay").drop("LateAircraftDelay")
    
#Drop TaxiIn and TaxiOut? This is suspicious. So just drop it.
data = data.drop("TaxiIn").drop("TaxiOut")

In [234]:
#IsArrDelayed and IsDepDelayed should be treated as boolean. enum is fine.
#Just focus on IsArrDelayed as in the lecture. Drop the other.
data = data.drop("IsDepDelayed")

In [235]:
#Construct a proper date
data["Date"] = (data["Year"]*10000) + (data["Month"]*100) + data["DayofMonth"]

In [236]:
#Recap. 
data.summary()
print data.shape

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,Origin,Dest,Distance,IsArrDelayed,Date
type,int,int,int,enum,int,int,enum,enum,enum,int,enum,enum,int,enum,int
mins,1987.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,11.0,0.0,19871001.0
mean,1997.5,1.40909090909,14.6010732639,NaN,798.380894993,902.699690754,NaN,NaN,NaN,125.021562607,NaN,NaN,730.182190565,0.555755150302,19975155.5102
maxs,2008.0,10.0,31.0,6.0,1439.0,1439.0,9.0,2438.0,3500.0,437.0,131.0,133.0,3365.0,1.0,20080103.0
sigma,6.34436090171,1.87471137134,9.17579042586,NaN,286.949414367,296.112371758,NaN,NaN,NaN,73.40159463,NaN,NaN,578.43800823,0.496887288343,63373.922583
zeros,0,0,0,5802,569,569,724,3,2,0,59,172,0,19537,0
missing,0,0,0,0,0,0,0,0,32,13,0,0,35,0,0
0,1987.0,10.0,14.0,3,450.0,529.0,PS,1451,NA,79.0,SAN,SFO,447.0,YES,19871014.0
1,1987.0,10.0,15.0,4,450.0,529.0,PS,1451,NA,79.0,SAN,SFO,447.0,YES,19871015.0
2,1987.0,10.0,17.0,6,450.0,529.0,PS,1451,NA,79.0,SAN,SFO,447.0,YES,19871017.0


(43978, 15)


### Weather data

In [310]:
#Check the size and summary of data
wthr.summary()
print wthr.shape

,Station,WBAN,Year,MonthDay,temp,temp cnt,dewpoint,dewpoint cnt,sea level pres,sea cnt,station pres,stat cnt,visibility,visi cnt,mean wind speed,wind speed cnt,max wind speed,gust speed,max temp,*is hourly max,min temp,*is hourly min,precipitation,precip report,snow depth,fog,rain,snow,hail,thunder,tornado
type,int,int,int,int,real,int,real,int,real,int,real,int,real,int,real,int,real,real,real,enum,real,enum,real,enum,real,int,int,int,int,int,int
mins,10010.0,102.0,2008.0,101.0,-113.0,4.0,-119.0,0.0,907.7,0.0,549.8,0.0,0.0,0.0,0.0,0.0,0.4,1.0,-106.2,0.0,-119.4,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0
mean,547314.964621,87463.0981285,2008.0,670.778050368,53.9973758002,16.8964896897,663.812227393,15.5385132786,4180.02237661,8.72217436396,4321.34239852,3.33330081324,256.850380356,12.0911487651,44.0299598744,16.1429810244,59.9926043587,747.738643225,72.2235527104,0.0,53.0420092554,0.0,9.32506548126,NaN,954.237926286,0.0573479912167,0.238952585991,0.0664748839505,0.00145052964359,0.0467688522362,0.000113581907003
maxs,999999.0,99999.0,2008.0,1231.0,110.0,24.0,9999.9,24.0,9999.9,24.0,9999.9,9.0,999.9,24.0,999.9,24.0,999.9,999.9,9999.9,0.0,9999.9,0.0,99.99,8.0,999.9,1.0,1.0,1.0,1.0,1.0,1.0
sigma,309885.125956,28861.3817425,0.0,345.063581283,22.8949965519,7.93400813534,2407.3711756,8.69842287125,4292.37877298,9.0275817175,4361.66342441,3.01801174662,428.287582024,9.85777130798,189.061972638,8.40498832878,211.997005667,427.20822904,300.865893076,0.0,279.380764318,0.0,28.9676708138,NaN,207.631492924,0.232506375367,0.426443782541,0.249110400242,0.0380581925706,0.211143409681,0.0106568775279
zeros,0,0,0,0,659,0,860,217897,0,1231437,0,1325705,3925,871534,37182,131514,0,0,853,1323391,1772,1638361,2377026,31011,0,3294828,2660070,3262927,3490205,3331805,3494878
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2171884,0,1856914,0,323728,0,0,0,0,0,0,0
0,10010.0,99999.0,2008.0,101.0,36.0,24.0,32.7,24.0,1004.7,24.0,1003.5,4.0,6.2,6.0,23.2,24.0,29.1,999.9,36.9,,33.4,,0.11,G,999.9,0.0,1.0,0.0,0.0,0.0,0.0
1,10010.0,99999.0,2008.0,102.0,33.3,24.0,29.3,24.0,1007.8,24.0,1006.6,4.0,5.7,6.0,9.6,24.0,23.3,999.9,35.8,*,29.1,*,0.06,G,999.9,0.0,1.0,0.0,0.0,0.0,0.0
2,10010.0,99999.0,2008.0,103.0,35.0,24.0,32.8,24.0,1017.5,24.0,1016.4,4.0,1.6,6.0,12.2,24.0,19.4,999.9,36.7,,28.8,,0.34,G,999.9,1.0,1.0,0.0,0.0,0.0,0.0


(3495275, 31)


General theme: specify data types correctly, do some conversion, and get rid of bad predictors. We also build a new feature called 'Date' to merge data in the future.

In [311]:
#Construct a proper date
wthr["Date"] = (wthr["Year"]*10000) + wthr["MonthDay"]

In [312]:
#Select only relevant columns
wthr = wthr[["Date","Station","temp","sea level pres","visibility",\
               "mean wind speed","precipitation","snow depth","fog","rain","snow","hail","thunder","tornado"]]

In [313]:
#Fix those columns with 9999.9 etc.
wthr[wthr["Station"]==999999.0,"Station"] = None
wthr[wthr["sea level pres"]==9999.9,"sea level pres"] = None
wthr[wthr["visibility"]==999.9,"visibility"] = None
wthr[wthr["mean wind speed"]==999.9,"mean wind speed"] = None
wthr[wthr["precipitation"]==99.99,"precipitation"] = None
wthr[wthr["snow depth"]==999.9,"snow depth"] = None

### Airport/Weather station link data

I have run basic checks on data as follows

In [241]:
#Check the size and summary of data
airport_weather_link.summary()
print airport_weather_link.shape

,C1,C2
type,enum,int
mins,0.0,4280.0
mean,NaN,508430.342789
maxs,6433.0,999120.0
sigma,NaN,297727.401584
zeros,1,0
missing,35602,21700
0,ADL,nan
1,,nan
2,BHQ,946890.0


(43591, 2)


General theme: select revelant columns. It turns out that we need only two of them: iata_xref (airpot name) and maslib (6-digit station code) 

Reference: http://weather.noaa.gov/tg/site.shtml

In [242]:
#No need for this as we already have two columns of what we want.
#airport_weather_link = airport_weather_link[['iata_xref','maslib']]

### Merging data

Here one may just use flight data to build a model. However, it might be helpful to integrate the weather information at the destination airport at the date. This leads to a scheme to merge data as follows. First, we merge flight data and airport/weather station link. Here we look for a weather station associated with the destination airport.

In [243]:
#Let's connect with weather information on that day at the arrival airport. We know what is what since we reformat the weather link file.
airport_weather_link.set_name("C1","Dest")
airport_weather_link.set_name("C2","Station")

In [248]:
#Merge with flight data
data_with_link = data.merge(airport_weather_link,all_x=True,all_y=False)

It is useful to select weather data related to relevant stations

In [343]:
wthr.shape

(3495275, 14)

In [363]:
wthr_with_link = wthr.merge(airport_weather_link,all_x=True,all_y=False)

In [366]:
wthr2 = wthr_with_link[~wthr_with_link["Dest"].isna()]
wthr2 = wthr2.drop("Dest")

In [367]:
wthr2.shape

(1096685, 14)

In [368]:
#Create the feature to merge
wthr2["DateStation"] = (wthr2["Date"]*1000000) + wthr2["Station"]
wthr2 = wthr2.drop("Station").drop("Date") # no longer needed these columns once merged

There are only 115 relevant stations!

Now let's merge flight data and weather data. We create a feature call "DateStation" as a key to merge two data tables.

In [369]:
#Create the feature to merge
data_with_link["DateStation"] = (data_with_link["Date"]*1000000) + data_with_link["Station"]
data_with_link = data_with_link.drop("Station") #Redundant information (similar to destination) 
data_with_link = data_with_link.drop("Date") #Redundant 

In [370]:
wthr2.shape, data_with_link.shape

((1096685, 12), (43978, 14))

In [371]:
#Merge data with DateStation key
data_with_weather = data_with_link.merge(wthr2,all_x=True,all_y=False)
data_with_weather = data_with_weather.drop("DateStation") #no longer need this link

## Stage 4: Model the data

In term of preparing data for validation, it is done in the function split_fit_predict as defined below. It is modified such that the peformance measure is AUC.

Here four models are used: GBM (Gradient Boost Method) DRF (Distributed Random Forest) GLM (Generalized Linear Model) and Deep Learning.

In [372]:
def split_fit_predict(data):
    global gbm0,drf0,glm0,dl0
    # Classic Test/Train split
    r = data['Year'].runif() # Random UNIForm numbers, one per row
    train = data[ r < 0.7]
    test = data[0.7 <= r]
    print("Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows")
    flight_names_x = data.names
    if "IsArrDelayed" in flight_names_x: flight_names_x.remove("IsArrDelayed")

    # Run GBM
    s = time.time()
    gbm0 = h2o.H2OGradientBoostingEstimator(ntrees=400, max_depth=6, learn_rate=0.1)
    gbm0.train(x=flight_names_x,y="IsArrDelayed",training_frame =train,validation_frame=test)
    gbm_elapsed = time.time() - s #measure elapse time

    # Run DRF
    s = time.time()
    drf0 = h2o.H2ORandomForestEstimator(ntrees=100, max_depth=30)
    drf0.train(x=flight_names_x,y="IsArrDelayed",training_frame =train,validation_frame=test)
    drf_elapsed = time.time() - s

    # Run GLM
    #if "WC1" in bike_names_x: bike_names_x.remove("WC1")
    s = time.time()
    glm0 = h2o.H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="binomial") #For logistic
    glm0.train(x=flight_names_x,y="IsArrDelayed",training_frame =train,validation_frame=test)
    glm_elapsed = time.time() - s

    # Run DL
    s = time.time()
    dl0 = h2o.H2ODeepLearningEstimator(hidden=[50,50,50,50], epochs=6)
    dl0.train(x=flight_names_x,y="IsArrDelayed",training_frame =train,validation_frame=test)
    dl_elapsed = time.time() - s

    # ----------
    # Score & report
    header = ["Model", "AUC TRAIN", "AUC TEST", "Model Training Time (s)"]
    table = [
     ["GBM", gbm0.auc(train=True), gbm0.auc(valid=True),
    round(gbm_elapsed,3)],
     ["DRF", drf0.auc(train=True), drf0.auc(valid=True),
    round(drf_elapsed,3)],
     ["GLM", glm0.auc(train=True), glm0.auc(valid=True),
    round(glm_elapsed,3)],
     ["DL ", dl0 .auc(train=True), dl0 .auc(valid=True),
    round( dl_elapsed,3)],
    ]
    h2o.display.H2ODisplay(table,header)
    # --------


First, let's look at the model without weather information at the destination airport.

In [373]:
# Split the data (into test & train), fit some models and look at the results
split_fit_predict(data)
# Explore (in Flow) the 4 models - training time, quality of fit, tendency to overfit


('Training data has', 15, 'columns and', 30861, 'rows, test has', 13117, 'rows')

gbm Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%

deeplearning Model Build Progress: [##################################################] 100%


Model,AUC TRAIN,AUC TEST,Model Training Time (s)
GBM,0.9180490,0.7494372,25.728
DRF,0.7386872,0.7529311,26.698
GLM,0.8151717,0.6941692,56.033
DL,0.7074555,0.6780372,56.402


Next, let's look at the model WITH weather information at the destination airport.

In [374]:
split_fit_predict(data_with_weather)

('Training data has', 26, 'columns and', 30781, 'rows, test has', 13197, 'rows')

gbm Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%

deeplearning Model Build Progress: [##################################################] 100%


Model,AUC TRAIN,AUC TEST,Model Training Time (s)
GBM,0.8526981,0.7363485,27.844
DRF,0.7369547,0.7435220,28.544
GLM,0.8178602,0.6838162,23.741
DL,0.7270701,0.6821493,61.612


By comparing results and checking model in H2O Flow (Go to http://localhost:54321/flow/index.html Choose Model > List All Models). I found that the best performing model is DRF WITHOUT weather information. Important features are Origin, Destination, and Flight number. Note that weather information may not be much helpful partly because the sample weather information hardly overlaps with the flight data. The difference may be due to random process. 

One way to improve the model is to add weather information at the origin airport well. But we will stop here.

Note that validation is done implicitly when we look at H2O Flow models and train-test comparison.

## Stage 5: Communicate the data

For small data, I concluded that potential methods are GBM and DRF. It is not clear yet if weather information useful. It can be useful when we run on the big data.

Here is an performance visual example from GBM with weather information (Note that numbers on figures may be different because figures are not upated in pace with the code run).

<img src="flight-delay-small-GBM-performance-1.JPG" width = "500x">

<img src="flight-delay-small-GBM-performance-2.JPG" width = "500x">


